In [1]:
import pandas as pd
import json
import numpy as np

# loading the lending club loans dataset

In [2]:
loans = pd.read_csv('lending-club-data.csv')
loans['safe_loans'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1)
loans = loans.drop('bad_loans', axis=1)

with open('module-5-assignment-2-train-idx.json','r') as f:
    train_idx_list = json.load(f)
train_idx = [int(i) for i in train_idx_list]

with open('module-5-assignment-2-test-idx.json', 'r') as f:
    test_idx_list = json.load(f)
test_idx = [int(i) for i in test_idx_list]

//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (19,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# data preparation

In [3]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'

loans = loans[features + [target]]

In [4]:
# one-hot encoding
loans_data = pd.get_dummies(loans[features], prefix=features, prefix_sep='.')
loans_data = pd.concat([loans_data, loans[target]], axis=1)

In [5]:
len(loans_data.columns[:-1]) # number of features (i.e. excl. target)

25

In [6]:
binary_features = list(loans_data.columns[:-1])
train_data = loans_data.iloc[train_idx]
test_data = loans_data.iloc[test_idx]

In [7]:
train_data.columns

Index(['grade.A', 'grade.B', 'grade.C', 'grade.D', 'grade.E', 'grade.F',
       'grade.G', 'term. 36 months', 'term. 60 months',
       'home_ownership.MORTGAGE', 'home_ownership.OTHER', 'home_ownership.OWN',
       'home_ownership.RENT', 'emp_length.1 year', 'emp_length.10+ years',
       'emp_length.2 years', 'emp_length.3 years', 'emp_length.4 years',
       'emp_length.5 years', 'emp_length.6 years', 'emp_length.7 years',
       'emp_length.8 years', 'emp_length.9 years', 'emp_length.< 1 year',
       'emp_length.n/a', 'safe_loans'],
      dtype='object')

In [8]:
binary_features

['grade.A',
 'grade.B',
 'grade.C',
 'grade.D',
 'grade.E',
 'grade.F',
 'grade.G',
 'term. 36 months',
 'term. 60 months',
 'home_ownership.MORTGAGE',
 'home_ownership.OTHER',
 'home_ownership.OWN',
 'home_ownership.RENT',
 'emp_length.1 year',
 'emp_length.10+ years',
 'emp_length.2 years',
 'emp_length.3 years',
 'emp_length.4 years',
 'emp_length.5 years',
 'emp_length.6 years',
 'emp_length.7 years',
 'emp_length.8 years',
 'emp_length.9 years',
 'emp_length.< 1 year',
 'emp_length.n/a']

# implementing a binary decision tree

In [49]:
class Btree_Model(object):
    '''
    An implementation of binary decision trees. Works only for binary features.
    An instance of the class is a model - in order to create the tree, use the create method, which returns the tree as a recursive dictionary. Hence, the instance (model) can be used to create several trees.
    Note that only the last tree is stored as an attribute.
    
    The model is equipped with classification and node count methods, but a tree must be passed to each.
    '''
    def __init__(self, max_depth=10, min_node_size=1, 
                 min_error_reduction=0.0):
        self.max_depth = max_depth
        self.tree = None
        self.min_node_size = min_node_size
        self.min_error_reduction = min_error_reduction
        print('A binary decision tree of max depth %d is initiated.' \
              %(self.max_depth))
        print('No further splits are allowed if a node has fewer than %d data points' \
             %self.min_node_size)
        print('As a stopping condition, we require that each split generate error reduction of at least %.1f' \
             %(self.min_error_reduction))
    
    @staticmethod
    def intermediate_node_num_mistakes(labels_in_node):
        if len(labels_in_node) == 0: return 0  
        labels_in_node = np.array(labels_in_node)
        num_positive = np.sum(labels_in_node==1)
        num_negative = np.sum(labels_in_node==-1)
        
        # Classification is by the majority rule, hence the minority class encompasses all mistakes
        num_mistakes = min(num_positive, num_negative)
        return num_mistakes
    
    @staticmethod
    def best_splitting_feature(data, features, target):
        assert type(features)==list and type(target)==str, "Features must be a list, and target must be a string."
        target_values = data[target]
        best_feature = None # Keep track of the best feature 
        best_error = 10     # Keep track of the best error so far
        
        # num_data_points = float(len(data)) # float division not an issue in Python 3
        
        for feature in features:
            # 1. Split on the binary feature
            left_split = data[data[feature] == 0]
            right_split = data[data[feature] == 1]
            
            # 2. Calculate the number of misclassified examples
            left_mistakes = Btree_Model.intermediate_node_num_mistakes(left_split[target])
            right_mistakes = Btree_Model.intermediate_node_num_mistakes(right_split[target])
            
            # 3. Calculate classification error for the split on this feature
            error = (left_mistakes + right_mistakes) / data.shape[0]
            
            # 4. Track the best feature
            if error <= best_error:
                best_feature = feature
                best_error = error
        return best_feature
        
    @staticmethod
    def create_leaf(target_values):
        '''
        Each node in the decision tree is represented as a dictionary.
        '''
        leaf = {'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf': True}
        
        num_ones = len(target_values[target_values == +1])
        num_minus_ones = len(target_values[target_values == -1])
        
        if num_ones > num_minus_ones:
            leaf['prediction'] = +1
        else:
            leaf['prediction'] = -1
        
        return leaf
    
    @staticmethod
    def error_reduction(error_before_split, error_after_split):
        '''
        computes the gain in error following a node split.
        '''
        return error_before_split - error_after_split
    
    def reached_minimum_node_size(self, data):
        return len(data) <= self.min_node_size
    
    def create(self, data, features, target, current_depth = 0):
        '''
        A recursive greedy algorithm to build the binary decision tree. 
        Max depth is stored as an attribute of the instance.
        '''
        self.target = target
        self.features = features
        remaining_features = list(features)
        target_values = data[target]
        print("--------------------------------------------------------------------")
        print("Subtree, depth = %s (%s data points)." % (current_depth, len(target_values)))
        
        # A recursive algorithm, hence we define stopping conditions
        
        if Btree_Model.intermediate_node_num_mistakes(target_values) == 0:
            print("Stopping condition 1 (single class node) reached.")     
            # If no mistakes at current node, make current node a leaf node
            return Btree_Model.create_leaf(target_values)
        
        if len(remaining_features) == 0:
            print("Stopping condition 2 (no more features) reached.")     
            return Btree_Model.create_leaf(target_values)
        
        if current_depth >= self.max_depth:
            print("Stopping condition 3 (max depth) reached.")     
            return Btree_Model.create_leaf(target_values)
        
        if self.reached_minimum_node_size(target_values):       
            print("Early stopping condition reached. Reached minimum node size.")
            return Btree_Model.create_leaf(target_values)
        
        # Determine best feature to split on
        splitting_feature = Btree_Model.best_splitting_feature(data, 
                                        remaining_features, target)
        
        # Split the node
        left_split = data[data[splitting_feature] == 0]
        right_split = data[data[splitting_feature] == 1]
        remaining_features.remove(splitting_feature)
        print("Split on feature %s. (%d, %d)" % (\
                      splitting_feature, len(left_split), len(right_split)))
        
        # Another stopping condition: don't split if insufficient error reduction
        error_before_split = Btree_Model.intermediate_node_num_mistakes(target_values) / len(data)
    
        left_mistakes = Btree_Model.intermediate_node_num_mistakes(left_split[target])
        right_mistakes = Btree_Model.intermediate_node_num_mistakes(right_split[target])
        error_after_split = (left_mistakes + right_mistakes) / len(data)
    
        if Btree_Model.error_reduction(error_before_split, 
                                       error_after_split) <= self.min_error_reduction: 
            print("Early stopping condition reached. Minimum error reduction.")
            return Btree_Model.create_leaf(target_values)
        
        # If a split generates an empty node (i.e., all data goes to either left or right)
        if len(left_split) == len(data):
            print("Creating leaf node.")
            return Btree_Model.create_leaf(left_split[target])
        if len(right_split) == len(data):
            print("Creating leaf node.")
            return Btree_Model.create_leaf(right_split[target])
        
        left_tree = self.create(left_split, remaining_features, 
                                target, current_depth+1)
        right_tree = self.create(right_split, remaining_features, 
                                 target, current_depth+1)
        
        self.tree = {'is_leaf'          : False, 
                    'prediction'       : None,
                    'splitting_feature': splitting_feature,
                    'left'             : left_tree, 
                    'right'            : right_tree}
        
        
        return self.tree
    
    def classify(self, tree, obs, annotate=False):
        '''
        Classifies an observation (array-like of inputs) by traversing the tree.
        '''
        if tree['is_leaf']:
            if annotate:
                print("At leaf, predicting %s" %(tree['prediction']))
            return tree['prediction']
        else:
            split_feature_value = obs[tree['splitting_feature']]
            if annotate:
                print("Split on %s = %s" % (tree['splitting_feature'], 
                                            split_feature_value))
            if split_feature_value == 0:
                return self.classify(tree['left'], obs, annotate)
            else:
                return self.classify(tree['right'], obs, annotate)
    
    def count_nodes(self, tree):
        if tree['is_leaf']:
            return 1
        return 1 + self.count_nodes(tree['left']) + self.count_nodes(tree['right'])
    
    def evaluate(self, tree, data):
        # calcuates classification error on data
        predictions = data.apply(lambda x: self.classify(tree, x), axis=1)
        accuracy = np.sum(predictions==data[self.target])/data.shape[0]
        return 1-accuracy

In [40]:
binary_dtree_model_new = Btree_Model(max_depth=6, min_node_size = 100, min_error_reduction=0.0)

A binary decision tree of max depth 6 is initiated.
No further splits are allowed if a node has fewer than 100 data points
As a stopping condition, we require that each split generate error reduction of at least 0.0


In [41]:
my_decision_tree_new = binary_dtree_model_new.create(train_data, binary_features, 'safe_loans')

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term. 60 months. (28001, 9223)
--------------------------------------------------------------------
Subtree, depth = 1 (28001 data points).
Split on feature grade.D. (23300, 4701)
--------------------------------------------------------------------
Subtree, depth = 2 (23300 data points).
Split on feature grade.E. (22024, 1276)
--------------------------------------------------------------------
Subtree, depth = 3 (22024 data points).
Split on feature grade.F. (21666, 358)
--------------------------------------------------------------------
Subtree, depth = 4 (21666 data points).
Split on feature emp_length.n/a. (20734, 932)
--------------------------------------------------------------------
Subtree, depth = 5 (20734 data points).
Split on feature grade.G. (20638, 96)
--------------------------------------------------------------------
Subtree, depth = 6 (20638 

In [62]:
print(test_data.iloc[0][target])
binary_dtree_model_new.classify(my_decision_tree_new, test_data.iloc[0], annotate=True)

-1.0
Split on term. 60 months = 1.0
Split on grade.A = 0.0
At leaf, predicting -1


-1

In [63]:
def print_stump(tree, name = 'root'):
    split_name = tree['splitting_feature'] # split_name is something like 'term. 36 months'
    if split_name is None:
        print("(leaf, label: %s)" % tree['prediction'])
        return None
    split_feature, split_value = split_name.split('.')
    print( '                       %s' % name)
    print( '         |---------------|----------------|')
    print( '         |                                |')
    print( '         |                                |')
    print( '         |                                |')
    print( '  [{0} == 0]               [{0} == 1]    '.format(split_name))
    print( '         |                                |')
    print( '         |                                |')
    print( '         |                                |')
    print( '    (%s)                         (%s)' \
        % (('leaf, label: ' + str(tree['left']['prediction']) if tree['left']['is_leaf'] else 'subtree'),
           ('leaf, label: ' + str(tree['right']['prediction']) if tree['right']['is_leaf'] else 'subtree')))

# evaluating classification error

In [64]:
binary_dtree_model_new.evaluate(my_decision_tree_new, test_data)

0.38367083153813009

In [65]:
binary_dtree_model_new.count_nodes(my_decision_tree_new)

21